In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import shutil
import zipfile
import glob
import os
import time

In [2]:
import os

path="/home/chohj/test_image/normal"
file_list=os.listdir(path)
test_n=[file for file in file_list if file.endswith(".jpg")] 

path="/home/chohj//test_image/unnormal"
file_list=os.listdir(path)
test_un=[file for file in file_list if file.endswith(".jpg")] 

In [3]:
path="/home/chohj/train_image/normal"
file_list=os.listdir(path)
train_n=[file for file in file_list if file.endswith(".jpg")] 

path="/home/chohj/train_image/unnormal"
file_list=os.listdir(path)
train_un=[file for file in file_list if file.endswith(".jpg")] 

In [4]:
path="/home/chohj/val_image/normal"
file_list=os.listdir(path)
valid_n=[file for file in file_list if file.endswith(".jpg")] 

path="/home/chohj/val_image/unnormal"
file_list=os.listdir(path)
valid_un=[file for file in file_list if file.endswith(".jpg")] 

In [5]:
class CustomDataset1(torch.utils.data.Dataset):
    def __init__(self, files, root, mode='train', transform=None):
        self.files = files
        self.root = root
        self.mode = mode
        self.transform=transform
        self.label = 0
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img = PIL.Image.open(os.path.join(self.root, self.files[index]))
        
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            return img, np.array([self.label])
        else:
            return img, self.files[index]

In [6]:
class CustomDataset2(torch.utils.data.Dataset):
    def __init__(self, files, root, mode='train', transform=None):
        self.files = files
        self.root = root
        self.mode = mode
        self.transform=transform
        self.label = 1
        
    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img = PIL.Image.open(os.path.join(self.root, self.files[index]))
        
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            return img, np.array([self.label])
        else:
            return img, self.files[index]

In [7]:
train_transform = torchvision.transforms.Compose([
torchvision.transforms.Resize((440,224)),
torchvision.transforms.ToTensor(),
])

test_transform = torchvision.transforms.Compose([
torchvision.transforms.Resize((440,224)),
torchvision.transforms.ToTensor(),
])

In [8]:
import pandas as pd

In [13]:
df1 = pd.read_csv('/home/chohj/train.csv')
df2 = pd.read_csv('/home/chohj/valid.csv')
df3 = pd.read_csv('/home/chohj/test.csv')

In [14]:
train_normal_dataset = CustomDataset1(train_n, '/home/chohj/train_image/normal', transform=train_transform)
train_unnormal_dataset = CustomDataset2(train_un, '/home/chohj/train_image/unnormal', transform=train_transform)
valid_normal_dataset = CustomDataset1(valid_n, '/home/chohj/val_image/normal', transform=test_transform)
valid_unnormal_dataset = CustomDataset2(valid_un, '/home/chohj/val_image/unnormal', transform=test_transform)
test_normal_dataset = CustomDataset1(test_n, '/home/chohj/test_image/normal', transform=test_transform)
test_unnormal_dataset = CustomDataset2(test_un, '/home/chohj/test_image/unnormal', transform=test_transform)
 
train_dataset = torch.utils.data.ConcatDataset([train_normal_dataset, train_unnormal_dataset])
valid_dataset = torch.utils.data.ConcatDataset([valid_normal_dataset, valid_unnormal_dataset])
test_dataset = torch.utils.data.ConcatDataset([test_normal_dataset, test_unnormal_dataset])

In [ ]:
tr_nor = os.listdir('/home/chohj/train_image/normal')
tr_unnor = os.listdir('/home/chohj/train_image/unnormal')

print('nor : {0} /  unnom:{1}'.format(len(tr_nor),len(tr_unnor)))

In [ ]:
print(f'number of train dataset : {len(train_dataset)}')
print(f'number of valid dataset : {len(valid_dataset)}')
print(f'number of test dataset : {len(test_dataset)}')

In [25]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
# 이미지 확인
samples, labels = iter(train_loader).next()
classes = {0:'normal', 1:'unnormal'}
fig = plt.figure(figsize=(16,24))
for i in range(24):
    a = fig.add_subplot(4,6,i+1)
    a.set_title(classes[labels[i].item()])
    a.axis('off')
    a.imshow(np.transpose(samples[i].numpy(), (1,2,0)))
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=0)

In [26]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [27]:
model = torchvision.models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
nn.Dropout(0.5),
nn.Linear(num_ftrs, 256),
nn.Tanh(),
nn.Dropout(),
nn.Linear(256, 1),
nn.Sigmoid()
)

# num_ftrs = model.fc.in_features
# model.fc = nn.Sequential(
# nn.Dropout(0.5),
# nn.Linear(num_ftrs, 1024),
# nn.Dropout(0.2),
# nn.Linear(1024, 512),
# nn.Dropout(0.1),
# nn.Linear(512, 1),
# nn.Sigmoid()
# )

In [29]:
from torchsummary import summary

model = model.to(device)
summary(model, input_size=(3,440,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 220, 112]           9,408
       BatchNorm2d-2         [-1, 64, 220, 112]             128
              ReLU-3         [-1, 64, 220, 112]               0
         MaxPool2d-4          [-1, 64, 110, 56]               0
            Conv2d-5          [-1, 64, 110, 56]          36,864
       BatchNorm2d-6          [-1, 64, 110, 56]             128
              ReLU-7          [-1, 64, 110, 56]               0
            Conv2d-8          [-1, 64, 110, 56]          36,864
       BatchNorm2d-9          [-1, 64, 110, 56]             128
             ReLU-10          [-1, 64, 110, 56]               0
       BasicBlock-11          [-1, 64, 110, 56]               0
           Conv2d-12          [-1, 64, 110, 56]          36,864
      BatchNorm2d-13          [-1, 64, 110, 56]             128
             ReLU-14          [-1, 64, 

In [30]:
def fit(model, criterion, optimizer, epochs, train_loader, valid_loader):

    model.train()

    train_loss = 0
    train_acc = 0
    train_correct = 0

    train_losses = []
    train_accuracies = []
    valid_losses = []
    valid_accuracies = []

    for epoch in range(epochs):
        start = time.time()
        for train_x, train_y in train_loader:
            model.train()
            train_x, train_y = train_x.to(device), train_y.to(device).float()
            optimizer.zero_grad()
            pred = model(train_x)
            loss = criterion(pred, train_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            y_pred = pred.cpu()
            y_pred[y_pred >= 0.5] = 1
            y_pred[y_pred < 0.5] = 0
            train_correct += y_pred.eq(train_y.cpu()).int().sum()
        # validation data check
        valid_loss = 0
        valid_acc = 0
        valid_correct = 0
        for valid_x, valid_y in valid_loader:
            with torch.no_grad():
                model.eval()
                valid_x, valid_y = valid_x.to(device), valid_y.to(device).float()
                pred = model(valid_x)
                loss = criterion(pred, valid_y)
            valid_loss += loss.item()

            y_pred = pred.cpu()
            y_pred[y_pred >= 0.5] = 1
            y_pred[y_pred < 0.5] = 0
            valid_correct += y_pred.eq(valid_y.cpu()).int().sum()

        train_acc = train_correct/len(train_loader.dataset)
        valid_acc = valid_correct/len(valid_loader.dataset)

        print(f'{time.time() - start:.3f}sec : [Epoch {epoch+1}/{epochs}] -> train loss: {train_loss/ len(train_loader):.4f}, train acc: {train_acc*100:.3f}% / valid loss: {valid_loss/len(valid_loader):.4f}, valid acc: {valid_acc*100:.3f}%')
    
        train_losses.append(train_loss/len(train_loader))
        train_accuracies.append(train_acc)
        valid_losses.append(valid_loss/len(valid_loader))
        valid_accuracies.append(valid_acc)

        train_loss = 0
        train_acc = 0
        train_correct = 0

    plt.plot(train_losses, label='loss')
    plt.plot(train_accuracies, label='accuracy')
    plt.legend()
    plt.title('train loss and accuracy')
    plt.show()
    plt.plot(valid_losses, label='loss')
    plt.plot(valid_accuracies, label='accuracy')
    plt.legend()
    plt.title('valid loss and accuracy')
    plt.show()

In [31]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
fit(model, criterion, optimizer, 100, train_loader, valid_loader)


18.710sec : [Epoch 1/100] -> train loss: 0.6929, train acc: 59.996% / valid loss: 0.7684, valid acc: 55.244%
18.532sec : [Epoch 2/100] -> train loss: 0.6588, train acc: 62.553% / valid loss: 0.6449, valid acc: 63.109%
18.457sec : [Epoch 3/100] -> train loss: 0.6592, train acc: 61.495% / valid loss: 0.6817, valid acc: 59.660%
18.740sec : [Epoch 4/100] -> train loss: 0.6576, train acc: 63.011% / valid loss: 0.6690, valid acc: 57.544%
18.683sec : [Epoch 5/100] -> train loss: 0.6562, train acc: 63.293% / valid loss: 0.6973, valid acc: 61.730%
18.324sec : [Epoch 6/100] -> train loss: 0.6556, train acc: 63.434% / valid loss: 0.7667, valid acc: 61.822%
18.464sec : [Epoch 7/100] -> train loss: 0.6640, train acc: 60.807% / valid loss: 0.6559, valid acc: 54.370%
19.531sec : [Epoch 8/100] -> train loss: 0.6503, train acc: 62.712% / valid loss: 0.6763, valid acc: 61.224%
19.149sec : [Epoch 9/100] -> train loss: 0.6622, train acc: 62.412% / valid loss: 0.6270, valid acc: 63.017%
19.177sec : [Epoch 

In [41]:
def eval(model, criterion, test_loader):
    with torch.no_grad():
        model.eval()
        correct = 0
        losses = 0
        t1 = time.time()
        for test_x, test_y in test_loader:
            test_x, test_y = test_x.to(device), test_y.to(device).float()
            pred = model(test_x)
            loss = criterion(pred, test_y)
            
            y_pred = pred.cpu()
            y_pred[y_pred >= 0.5] = 1
            y_pred[y_pred < 0.5] = 0

            losses += loss.item()
            correct += y_pred.eq(test_y.cpu()).int().sum()
            t2 = time.time()
            infer_time = (t2 - t1) * 1000
            print(f'eval loss: {losses/len(test_loader):.4f}, eval acc: {correct/len(test_loader.dataset)*100:.3f}%')
            print(infer_time)
        print('all time : ', (time.time - t1)*1000 / 1941)

In [42]:
eval(model, criterion, test_loader)

eval loss: 0.0070, eval acc: 1.340%
eval loss: 0.0155, eval acc: 2.421%
eval loss: 0.0250, eval acc: 3.606%
eval loss: 0.0302, eval acc: 5.049%
eval loss: 0.0382, eval acc: 6.285%
eval loss: 0.0455, eval acc: 7.419%
eval loss: 0.0511, eval acc: 8.913%
eval loss: 0.0577, eval acc: 10.252%
eval loss: 0.0657, eval acc: 11.540%
eval loss: 0.0724, eval acc: 12.880%
eval loss: 0.0830, eval acc: 13.910%
eval loss: 0.0916, eval acc: 15.095%
eval loss: 0.0979, eval acc: 16.538%
eval loss: 0.1047, eval acc: 17.877%
eval loss: 0.1132, eval acc: 19.011%
eval loss: 0.1185, eval acc: 20.453%
eval loss: 0.1263, eval acc: 21.587%
eval loss: 0.1345, eval acc: 22.772%
eval loss: 0.1433, eval acc: 23.957%
eval loss: 0.1495, eval acc: 25.348%
eval loss: 0.1563, eval acc: 26.636%
eval loss: 0.1631, eval acc: 27.975%
eval loss: 0.1711, eval acc: 29.263%
eval loss: 0.1787, eval acc: 30.551%
eval loss: 0.1863, eval acc: 31.736%
eval loss: 0.1932, eval acc: 33.127%
eval loss: 0.2018, eval acc: 34.312%
eval los